In [1]:
import os
import csv
import transformers
from io import StringIO
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
import torch
#import requests
torch.manual_seed(42)

In [ ]:
flanT5tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
flanT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")
flanT5.eval()

In [ ]:
OHNCTokenizer = AutoTokenizer.from_pretrained("Weyaxi/neural-chat-7b-v3-1-OpenHermes-2.5-7B")
OHNC = AutoModelForCausalLM.from_pretrained("Weyaxi/neural-chat-7b-v3-1-OpenHermes-2.5-7B")

In [ ]:
mixtralTokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
Mixtral = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [ ]:
##Platypus
platypusTokenizer = AutoTokenizer.from_pretrained("garage-bAInd/Platypus2-7B")
Platypus = AutoModelForCausalLM.from_pretrained("garage-bAInd/Platypus2-7B")

In [ ]:
def setParameters(newLevels, newModel, newPromptAdaption):
    global levels, model, promptAdaption
    levels = newLevels
    model = newModel
    promptAdaption = newPromptAdaption


levels = 5
model = "flanT5"
promptAdaption = False




setParameters(5, model, False)
#Counter
startcount=1
endcount=-1




In [ ]:
def getAnswer(prompt, model, systemPrompt):
    
    
    ret="No model defined!"
    
    if model=="OHNC":        
        inputs = OHNCTokenizer(prompt+systemPrompt, return_tensors="pt")
        input_ids = inputs["input_ids"]
        outputs = OHNC.generate(input_ids, 
                                max_new_tokens=400, 
                                pad_token_id=OHNCTokenizer.eos_token_id,
                                do_sample=False)
        ret = OHNCTokenizer.decode(outputs[0])#, skip_special_tokens=True)
        ret = ret[len(prompt+systemPrompt):len(ret)]
    elif model=="flanT5":
        input_ids = flanT5tokenizer(prompt+systemPrompt, return_tensors='pt').input_ids.to("cuda")
        outputs = flanT5.generate(input_ids, 
                                max_new_tokens=20,
                                num_beams=1)
        ret=flanT5tokenizer.decode(outputs[0])
    elif model=="mixtral":
        inputText="<s>[INST] {"+prompt+systemPrompt+"}[/INST]"
        inputs = mixtralTokenizer(inputText, return_tensors="pt")
        outputs = Mixtral.generate(**inputs,
                                   max_new_tokens=400)
        ret=mixtralTokenizer.decode(outputs[0])
        ret = ret[len(inputText)+4:len(ret)]
    elif model =="platypus":    
        inputs = platypusTokenizer(prompt, return_tensors="pt")
        outputs = Platypus.generate(**inputs,
                                    temperature=0,
                                    max_new_tokens=20, 
                                    num_return_sequences=1)

        ret = platypusTokenizer.decode(outputs[0])
        ret = ret[len(prompt)+4:len(ret)]
    if(ret=="No model defined!"):
        print(ret)   
    return ret

In [ ]:


def testRun(newLevels, newModel, newPromptAdaption, dataset):
    setParameters(newLevels, newModel, newPromptAdaption)
    if dataset=="CDSS_in_RO":
        startcount=1
        endcount=-1
        relevantCriteria="""
        -Only inclusion of original articles, exclusion of review articles
        -Publications examining one or several clinical decision-support systems relevant for radiation therapy
        -Decision-support systems are software based
        -Exclusion of systems intended for support of non-clinicians (e.g. patient decision-aids)
        -Publications about models (e.g. prognostic models) should only be included, if the model is intended to support clinical decision-making as part of a software application, which may resemble a clinical decision support system
        """
    elif dataset=="Appenzeller-Herzog":
        startcount=1
        endcount=-1
        relevantCriteria="""
        -Patients with Wilson's Disease of any age or stage 
        -Study drug has to be one of four established therapies, namely DPen, trientine, TTM or Zn.
        -Control could be placebo, no treatment or any other treatment that does not include the respective study drug 
        -Concomitant therapies had to be identical in the compared treatment arms
        -Combination therapy regimens that include the respective monotherapy drug are not considered
        -Prospective or retrospective studies reported 
        -Randomized, non-randomized controlled trials and comparative observational studies
        -Animal studies, case reports, case series, cross‐sectional studies, before‐after studies, reviews, letters, abstract‐only publications, editorials, 
        diagnostic or other testing studies and non‐controlled studies are excluded
        """
    elif dataset=="Wolters":
        startcount=1
        endcount=-1
        relevantCriteria="""
        -Prospective studies of humans that report the risk of all-cause dementia or Alzheimer's disease in relation to coronary heart disease or congestive heart failure
        -Only original articles
        """
    elif dataset=="Bos_2018":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Inclusion criteria: (1) all studies have to be prospective
        population-based cohort studies. Inherent to the nature of
        population-based studies this means that except for a
        selection criterion of age, no other selection criteria are
        applied; (2) cerebral imaging had to be performed
        (either magnetic resonance imaging [MRI] or computed
        tomography) for the visualization of white matter
        hyperintensities, covert brain infarcts, or microbleeds; (3) all
        studies have to have investigated the association of any of the
        three pathologies with the risk of incident all-dementia or
        Alzheimer’s disease.
        """
    elif dataset=="Brouwer_2019":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Criteria for studies to be included in the review are: (1) Presence
        of a diagnostic status of major depressive disorder (MDD) (MDD absence and/or presence) for all
        participants, as determined through a clinical interview or by a clinician at the start of the
        study; (2) At some point during the study, participants needed to be in
        (partial) remission or recovery as determined by a clinical interview or
        clinician assessment; and (3) relapse or recurrence was diagnosed
        through a clinical interview or by a clinician. (4) The study design was
        longitudinal and prospective; (5) The theory-derived predictor (i.e., the
        proposed factors) was assessed before the relapse or recurrence of MDD;
        (6) The predictor was derived from one of the leading psychological
        theories; (7) Sufficient information was reported to calculate effect sizes
        (or was made available upon request). Exclusion criteria are the
        presence of bipolar disorder, dysthymia, seasonal affective disorder,
        postpartum depression, late-life MDD, or MDD due to medical disorders. Studies that solely included people with a first onset MDD when
        they were older than 65 years, were excluded due to potential etiological differences between late-life onset MDD and MDD at younger ages
        """
    elif dataset=="Donners_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        The following inclusion criteria are applied: emicizumab
        studies providing (1) data on humans, (2) original PK data or
        modeled PK data or PK/PD relationships, and (3) access to
        the abstract and the full text in English. In the event of doubt
        regarding eligibility, the records or articles should be included.
        """
    elif dataset=="Jeyaraman_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Inclusion Criteria
        Studies are included if they meet the following PICOS criteria:
        Population: Patients with knee osteoarthritis
        Intervention: MSC therapy
        Comparator: Usual care
        Outcomes: Visual Analog Score (VAS) for Pain, Western Ontario McMaster Universities Osteoarthritis Index (WOMAC), Lysholm Knee Scale (Lysholm), Whole-Organ Magnetic Resonance Imaging Score (WORMS), Knee Osteoarthritis Outcome Score (KOOS), and adverse events
        Study Design: Randomized controlled trials
        Exclusion Criteria
        Trials are excluded if they had the following characteristics:
        1. Observational studies and interventional studies without a comparator group
        2. Animal studies involving stem cell therapy for knee osteoarthritis models
        3. Review
        """
    elif dataset=="Leenaars_2019":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Exclusion of (1) studies on other techniques than microdialysis
        (e.g. biosensors and microdialysis precursors such
        as push-pull perfusion), (2) studies measuring other substances
        than Hist and the amino acids Asn, Asp, GABA,
        Glu, Gln, Gly, Pro and Tau, (3) retro-dialysis studies, (4)
        microdialysis studies that did not report baseline values
        without the specified molecules in the perfusion fluid,
        (5) extra-cerebral microdialysis studies, (6) human and in
        vitro studies, and (7) papers not containing primary study
        data. Within publications, experiments using techniques
        other than microdialysis are also ignored, as
        well as data on amino acids other than those searched for.
        During screening, tags were added by KJ and CL to all
        studies on circadian rhythms, sleep and sleep deprivation.
        Studies on circadian rhythms, sleep and sleep deprivation are included based on the following criterion: 
        studies measuring one
        or more of the molecules of interest during (1) naturally
        occurring sleep stages that were validated with polysomnographic
        measurements and/or (2) during sleep deprivation.
        Studies on e.g. carbachol-induced REM-sleep are
        thus excluded.
        """
    elif dataset=="Leenaars_2020":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Inclusion of all studies in which methotrexat (MTX) was administered to rheumatoid arthritis (RA) patients or RA animal models. 
        Exclusion of studies on other diseases than RA, studies that were not in vivo,
        studies that did not administer MTX (or only had treatment groups co-administering MTX with other
        experimental drugs), studies that did not analyse efficacy (e.g., safety studies), and publications that
        do not contain new data (e.g., reviews and editorials).
        """
    elif dataset=="Meijboom_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Articles are included if they meet the
        following criteria: (1) study involved transitioning from a
        TNF-alpha inhibitor (including etanercept, infliximab, and adalimumab)
        originator to a biosimilar, (2) the number of patients
        who retransitioned is reported or can be calculated, (3)
        the article is an original research article published in a
        peer-reviewed journal, (4) the article included baseline
        characteristics of the patients who transitioned, (5) the article
        is written in English. Transitioning is defined as
        patients in whom the biosimilar was introduced after the 
        originator, without treatment with other drugs in between.
        Retransitioning is defined as restarting the originator
        directly after discontinuing a biosimilar, without treatment
        with other drugs in between. In summary, transitioning is
        defined as switching from the originator to a biosimilar;
        retransitioning is defined as switching from the originator
        to a biosimilar and back to the originator. Both transitioning
        and retransitioning involve changes with the same active
        biological substance.
        """
    elif dataset=="Moran_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Original experimental studies
        that manipulated the condition of animals in independent
        treatment groups through their diet, via both dietary quantity
        (i.e. partial restriction, complete deprivation or enrichment)
        or quality treatments (e.g. protein restriction or enrichment),
        and including both short-term and longer-term manipulations
        up to extended periods of weeks to months. Studies that then subjected those animals to
        behavioural observations in contexts relating to risk
        (e.g. novel environments, novel object, risk-sensitive foraging,
        predator response) in independent trials.
        Inclusion criteria (all need to be true):
        -Topic is in a potentially relevant field (e.g., biology, ecology, psychology)
        -Paper contains original empirical data
        -Study species is a non-human, non-genetically modified animal
        -Study includes an experimental manipulation of animal´s nutritional or energetic state
        -Dietary manipulations (a) in the current generation, (b) not limited only to specific compounds/chemicals (e.g., omega-3 fatty acids), and (c) not limited to temporal/spatially variability in food
        -Behavioural response variables that are related to risk (e.g., predator response, novel environment/object, risk sensitive foraging)
        """
    elif dataset=="Muthu_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Inclusion Criteria
        To be included, a study should meet the
        following criteria:
        1. The study should be an RCT with 1:1 parallel twoarm
        design.
        2. The study must be related to spine surgery involving
        preoperative or intraoperative or postoperative variables.
        3. The study must have a dichotomous primary or
        secondary outcome.
        Exclusion Criteria
        1. Studies not involving human subjects.
        2. Studies with continuous variable outcomes like pain
        scores, Oswestry Disability Index scores, time to
        union without predefined clinical success criteria.
        3. Studies that did not report a statistically significant
        primary or secondary outcome measure.
        """
    elif dataset=="Oud_2018":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Randomized controlled trials (RCT)s on four specialized psychotherapies
        (DBT: dialectic behavior therapy, MBT: mentalization-based treatment, TFP: transference-focused therapy and ST: schema therapy) 
        for adults (18 years and older)
        with Borderline personality disorder (BPD), which includes an individual psychotherapy
        component and had a duration of 16 weeks or more.
        Eligible comparison groups are other protocolized and
        specialized psychotherapies, or control groups, for
        example, treatment as usual (TAU), waiting list, attention
        control or community treatment by experts (CTBE).
        Studies are excluded with a cut-off of <66% of the participants
        having BPD, unless disaggregated data are provided. Studies are excluded that tested incomplete
        versions of specialized treatment, for example,
        studies that investigated only skills training instead of
        the full DBT program.
        """
    elif dataset=="van_de_Schoot_2018":
        startcount=1
        endcount=-1
        relevantCriteria="""
        (a) longitudinal studies with at least three measurement
        waves measuring posttraumatic stress disorder (PTSD), (b) studies that measured PTSD on a
        continuous scale via an interviewor questionnaire, (c) and studies
        that used a clustering method (Latent growth mixture modelling, hierarchical
        cluster analysis), (d) traumatic stress symptoms following events
        that appeared to fulfill DSM-IV criterion A1 for PTSD or acute
        stress disorder.
        """
    elif dataset=="van_der_Valk_2021":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Studies that report original Long-term glucocorticoids (HairGC) data.
        """
    elif dataset=="van_Dis_2020":
        startcount=1
        endcount=-1
        relevantCriteria="""
        Randomized clinical trials are included that examined effects of cognitive behavioral therapy 
        (CBT; ie, any therapy with cognitive restructuring and/or a behavioral therapy, such as exposure, as core component),
        including third generation CBTs (ie, acceptance and commitment therapy and metacognitive therapy), at least 1 month after treatment completion,
        in an individual, group, or internet treatment format. Comparison groups include care as usual
        (ie, anything patients would normally receive as long as it was not a structured type of psychotherapy, such as primary care at 
        medical centers or case management with educational groups),15 relaxation, psychoeducation, pill placebo, supportive therapy, or 
        waiting list. Studies are included if they tested adult patients (or samples consisting mostly of adults but also some adolescents 
        aged ≥16 years) who received a diagnosis of generalized anxiety disorder, agoraphobia, social anxiety disorder, specific phobia, posttraumatic stress disorder, 
        or obsessive-compulsive disorder based on results of a structured diagnostic interview.
        Studies are excluded if they do not use CBT (eg, applied relaxation, eye movement desensitization and reprocessing, or interpersonal therapy) 
        or do not report symptoms separately for each disorder. To reduce clinical heterogeneity, studies were also excluded if they had done any of 
        the following: (1) used self-guided therapy without any guidance, (2) used CBT combined with medication or pill placebo, or (3) tested inpatients.
        """
    saveName = "result_L"+str(levels)+"_M"+model+"_PA"+str(promptAdaption)



    with open("dataset("+dataset+").csv", 'r', encoding='ISO-8859-1') as file:
        reader = csv.reader(file, delimiter=';')
        readerlist = list(reader)
        count = 0
        includedCount = 0
        notIncludedCount = 0
        eval = []         
        for i in range(levels - 1):
            eval.append([0,0,0,0,0])
        unclearCount=0
        resultRow=[]
        digitString = str(levels - 1) + " or " + str(levels) # shall be "1, 2, 3, 4 or 5" in the end
        for i in range(levels - 2, 0, -1):
            digitString = str(i) + ", " + digitString


        systemPrompt="You evaluate the relevance of scientific publications based on named criteria and based on title and abstract of a publication. Your answer consists only of a single digit (" + digitString + "). Do not include further explanation."

        highScoreText = str(levels) + ""
        if levels >= 5:
            highScoreText = str(levels - 1) + " or " + str(levels)
        if levels >= 10:
            highScoreText = str(levels - 2) + ", " + str(levels - 1) + " or " + str(levels)

        instruction="On a scale from 1 (very low probability) to " + str(levels) + " (very high probability), how would you rate the relevance of the scientific publication for inclusion into a systematic literature review based on the relevant criteria and based on title and abstract?"
        if promptAdaption==True:
            instruction=instruction+"(Note: Give a low score if not all criteria are fulfilled. Give only a high score if all or almost all criteria are fulfilled.)"


        count -= 1 #reduce count by one because it is increased right at the start of the for loop (instead of in the end because we have a 'continue' in the code)
        for row in readerlist:

            count+=1
            if count > endcount and endcount != -1: #if endcount is -1, there is no endcount
                break

            if count>=startcount:
                if count!=0:
                    title = row[0]
                    abstract = row[1]
                    included_label = row[2]
                    #Scale prompt
                    prompt=instruction
                    prompt=prompt+" Title: "+title+", Abstract: "+abstract+", Relevant criteria: "+relevantCriteria
                    
                    print("Count:"+str(count))
                    sumOfAnswers = 0
                    validRuns = 0
                    individualAnswers = []
                    error=True


                    if model=="OHNC" or model=="flanT5" or model=="mixtral" or model=="platypus":
                        answer = getAnswer(prompt,model,systemPrompt)
                        print("Answer:"+answer)
                        llmresponse=answer
                        regexFilter=re.search(r'\d+', answer)
                        if isinstance(regexFilter,type(None)):
                            answer=-1
                        else:
                            answer=re.search(r'\d+', answer).group()
                            answer=int(answer)
                        print("Result:"+str(answer))


                    if answer < 1 or answer > levels:
                        answer = -1


                    if answer >= 1 and answer <= levels:
                        sumOfAnswers += int(answer)
                        validRuns += 1
                    
                if validRuns == 0:
                    unclearCount += 1 #otherwise no answer was found and it is skipped
                    print("Skipped '" + title + "' because the answer was invalid.")
                        #continue
                else:
                    answer = float(sumOfAnswers / validRuns) #compute average answer

                singleResult=[]
                singleResult.append(str(count))
                singleResult.append(title)
                singleResult.append(abstract)
                singleResult.append(included_label)
                singleResult.append(answer)
                singleResult.append(llmresponse)
                for i in individualAnswers:
                    singleResult.append(i)



                with open(saveName+"(single).csv",'a',newline='',encoding='utf-8') as f:
                    writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
                    writer.writerow(singleResult)


                resultRow.append(singleResult)

                
                if included_label=="1":  
                    includedCount+=1

                    for i in range(levels - 1):
                        if (levels - i == answer):
                            eval[i][0] += 1 #increase counter
                        if (levels - i <= answer):
                            eval[i][1] += 1 #correctly included
                        else:
                            eval[i][4] += 1 #incorrectly included

                else:
                    notIncludedCount+=1

                    for i in range(levels - 1):
                        if (levels - i == answer):
                            eval[i][0] += 1 #increase counter
                        if (levels - i <= answer):
                            eval[i][2] += 1 #falsely included
                        else:
                            eval[i][3] += 1 #correctly not included
            

    # [number of counts, correctlyIncluded, falselyIncluded, correctlyNotIncluded, falselyNotIncluded]
    print("Included Publications: ",includedCount)

    for i in range(levels - 1):
        print("Totally Included (answer="+str(levels-i)+"): ",str(eval[i][0]))
        print("Correctly Included (answer="+str(levels-i)+"): ",str(eval[i][1]))
        print("Correctly Not Included (answer="+str(levels-i)+"): ",str(eval[i][3]))
        print("Falsely Included (answer="+str(levels-i)+"): ",str(eval[i][2]))
        print("Falsely Not Included (answer="+str(levels-i)+"): ",str(eval[i][4]))

    print("UnclearCount: ",unclearCount)


    # init evaluation arrays
    tp = []
    sensitivity = []
    precision = []
    recall = []
    f1 = []
    specifity = []
    for i in range(levels - 1):
        tp.append(0)
        sensitivity.append(0)
        precision.append(0)
        recall.append(0)
        f1.append(0)
        specifity.append(0)

    # do evaluation
    if includedCount>0:
        #tp=true positive only for this score
        prevEval1 = 0
        for i in range(levels - 1):
            if eval[i][0] > 0:
                tp[i] = (eval[i][1] - prevEval1) / eval[i][0]
            else:
                tp[i] = -1
            prevEval1 = eval[i][1]

            sensitivity[i] = eval[i][1] / includedCount

            if (eval[i][1] + eval[i][2]) > 0:
                precision[i] = eval[i][1] / (eval[i][1]+eval[i][2])
            else:
                precision[i] = -1
            
            if (eval[i][1] + eval[i][4]) > 0:
                recall[i] = eval[i][1] / (eval[i][1] + eval[i][4])
            else:
                recall[i] = 1
            
            if (precision[i] + recall[i]) > 0:
                f1[i] = 2 * ((precision[i] * recall[i]) / (precision[i] + recall[i]))
            else:
                f1[i] = -1
            
            specifity[i] = eval[i][3] / notIncludedCount
    else:
        for i in range(levels - 1):
            sensitivity[i] = -1
            precision[i] = -1
            recall[i] = -1
            f1[i] = -1
            specifity[i] = -1



    for i in range(levels - 1):
        print("True Positive (answer="+str(levels-i)+"): ",str(tp[i]))
        print("Sensitivity (answer="+str(levels-i)+"): ",str(sensitivity[i]))
        print("Specificity (answer="+str(levels-i)+"): ",str(specifity[i]))
        print("Precision (answer="+str(levels-i)+"): ",str(precision[i]))
        print("Recall (answer="+str(levels-i)+"): ",str(recall[i]))
        print("F1-Score (answer="+str(levels-i)+"): ",str(f1[i]))


    #Add empty line for results
    resultRow.append([])

    singleResult=[]
    singleResult.append("System Prompt")
    singleResult.append(systemPrompt)
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("Instruction Prompt")
    singleResult.append(instruction)
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("relevantCriteria Prompt")
    singleResult.append(relevantCriteria)
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("Levels:")
    singleResult.append(levels)
    resultRow.append(singleResult)




    #Add empty line for results
    resultRow.append([])


    singleResult=[]
    singleResult.append("Included Publications")
    singleResult.append(includedCount)
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("Correctly Included")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(eval[i][1])
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("Correctly NotIncluded")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(eval[i][3])
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("Falsely Included")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(eval[i][2])
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("Falsely NotIncluded")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(eval[i][4])
    resultRow.append(singleResult)



    singleResult=[]
    singleResult.append("Unclear Count")
    singleResult.append(unclearCount)
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("True positive")
    for i in range(levels, 1, -1):
        singleResult.append(str(i))
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(tp[i])
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("Sensitivity")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(sensitivity[i])
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("Specificity")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(specifity[i])
    resultRow.append(singleResult)


    singleResult=[]
    singleResult.append("Precision")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(precision[i])
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("Recall")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(recall[i])
    resultRow.append(singleResult)

    singleResult=[]
    singleResult.append("F1-Score")
    singleResult.append(str(levels))
    for i in range(levels - 1, 1, -1):
        singleResult.append(str(i) + "+")
    resultRow.append(singleResult)
    singleResult=[]
    singleResult.append("")
    for i in range(levels - 1):
        singleResult.append(f1[i])
    resultRow.append(singleResult)




    with open(saveName+".csv",'a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        header=[]
        header.append("No")
        header.append("Title")
        header.append("Abstract")
        # header.append("DOI")
        header.append("Included-Label")
        header.append("Answer of LLM")
        header.append("LLM-Response")

        writer.writerow(header)
        writer.writerows(resultRow)

In [11]:
#testRun(5, model, False, "CDSS_in_RO")

Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


Count:1
Answer:<pad> 4</s>
Result:4
Count:2
Answer:<pad> 4</s>
Result:4
Count:3
Answer:<pad> 4</s>
Result:4
Count:4
Answer:<pad> 4</s>
Result:4
Count:5
Answer:<pad> 4</s>
Result:4
Count:6
Answer:<pad> 4</s>
Result:4
Count:7
Answer:<pad> 4</s>
Result:4
Count:8
Answer:<pad> 4</s>
Result:4
Count:9
Answer:<pad> 4</s>
Result:4
Count:10
Answer:<pad> 4</s>
Result:4
Count:11
Answer:<pad> 4</s>
Result:4
Count:12
Answer:<pad> 4</s>
Result:4
Count:13
Answer:<pad> 4</s>
Result:4
Count:14
Answer:<pad> 4</s>
Result:4
Count:15
Answer:<pad> 4</s>
Result:4
Count:16
Answer:<pad> 4</s>
Result:4
Count:17
Answer:<pad> 4</s>
Result:4
Count:18
Answer:<pad> 4</s>
Result:4
Count:19
Answer:<pad> 4</s>
Result:4
Count:20
Answer:<pad> 4</s>
Result:4
Count:21
Answer:<pad> 4</s>
Result:4
Count:22
Answer:<pad> 4</s>
Result:4
Count:23
Answer:<pad> 4</s>
Result:4
Count:24
Answer:<pad> 4</s>
Result:4
Count:25
Answer:<pad> 4</s>
Result:4
Count:26
Answer:<pad> 4</s>
Result:4
Count:27
Answer:<pad> 4</s>
Result:4
Count:28
A

KeyboardInterrupt: 